In [1]:
import dgl.nn as dglnn
import dgl
import dgl.nn.functional as fn
import torch
import torch.nn as nn
import numpy as np
from dgl.nn import HeteroEmbedding
device = 'cpu'      # change to 'cuda' for GPU

graph = dgl.load_graphs("./graphs/industrial_and_scientific_5_core_new.dgl")[0][0]

train_nids = graph.ndata['train_split']
test_nids = graph.ndata['test_split']

In [2]:
graph.canonical_etypes

[('Brand', 'rev_SOLD_BY', 'Product'),
 ('Customer', 'WROTE', 'Review'),
 ('Product', 'SOLD_BY', 'Brand'),
 ('Product', 'rev_REVIEW_OF', 'Review'),
 ('Review', 'REVIEW_OF', 'Product'),
 ('Review', 'rev_WROTE', 'Customer')]

In [3]:
#sampler = dgl.dataloading.MultiLayerFullNeighborSampler(2)
sampler = dgl.dataloading.NeighborSampler([5, 10])

In [4]:
class StochasticTwoLayerRGCN(nn.Module):
    def __init__(self, emb_types, emb_size, hid_feats, out_feats, rel_names):
        super().__init__()
        self.embed = HeteroEmbedding(emb_types, emb_size)
        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(emb_size, hid_feats)
            for rel in rel_names}, aggregate='mean')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='mean')

    def forward(self, blocks):
        x = self.embed(blocks[0].ndata["_ID"])
        x = self.conv1(blocks[0], x)
        x = self.conv2(blocks[1], x)
        return x

In [5]:
class ScorePredictor(nn.Module):
    def forward(self, edge_subgraph, x):
        with edge_subgraph.local_scope():
            edge_subgraph.ndata['x'] = x
            for etype in edge_subgraph.canonical_etypes:
                edge_subgraph.apply_edges(
                    dgl.function.u_dot_v('x', 'x', 'score'), etype=etype)
            return edge_subgraph.edata['score']

class Model(nn.Module):
    def __init__(self, emb_types, in_features, hidden_features, out_features,
                 etypes):
        super().__init__()
        self.rgcn = StochasticTwoLayerRGCN(
            emb_types, in_features, hidden_features, out_features, etypes)
        self.pred = ScorePredictor()

    def forward(self, positive_graph, negative_graph, blocks):
        x = self.rgcn(blocks)
        pos_score = self.pred(positive_graph, x)
        neg_score = self.pred(negative_graph, x)
        return pos_score, neg_score

In [6]:
train_eid_dict = {
    etype: graph.edges(etype=etype, form='eid')
    for etype in graph.canonical_etypes}

sampler = dgl.dataloading.as_edge_prediction_sampler(
    sampler, negative_sampler=dgl.dataloading.negative_sampler.PerSourceUniform(5))

dataloader = dgl.dataloading.DataLoader(
    graph, train_eid_dict, sampler,
    batch_size=2048,
    shuffle=True,
    drop_last=False,
    num_workers=1)

In [7]:
def compute_loss(pos_score, neg_score):
    # an example hinge loss
    loss = 0
    num_summed = 0
    for e_type in pos_score.keys():
        n = pos_score[e_type].shape[0]
        if n > 0:
            loss += (neg_score[e_type].view(n, -1) - pos_score[e_type].view(n, -1) + 1).clamp(min=0).mean()
            num_summed += 1
    return loss / num_summed

In [8]:
model = Model({x: graph.number_of_nodes(x) for x in graph.ntypes}, 512, 256, 256, graph.etypes)
model = model.to(device)
opt = torch.optim.Adam(model.parameters())

epochs = 5

for i in range(epochs):
    epoch_loss = 0
    for input_nodes, positive_graph, negative_graph, blocks in dataloader:
        blocks = [b.to(device) for b in blocks]
        positive_graph = positive_graph.to(device)
        negative_graph = negative_graph.to(device)
        pos_score, neg_score = model(positive_graph, negative_graph, blocks)
        loss = compute_loss(pos_score, neg_score)
        epoch_loss += loss
        opt.zero_grad()
        loss.backward()
        opt.step()
    epoch_loss /= len(dataloader)
    print("=== EPOCH: {} \t LOSS: {}".format(i, epoch_loss))


/opt/anaconda3/envs/huggingface/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [33]:
X = model.rgcn.embed.embeds["Review"](torch.tensor([i for i in range(graph.num_nodes("Review"))])).detach().numpy()
y = graph.ndata["Positive"]["Review"].detach().numpy()

In [34]:
X.shape

(77060, 512)

In [35]:
y.shape

(77060,)

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [40]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(class_weight="balanced")
clf.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

In [41]:
from sklearn.metrics import classification_report

print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.11      0.45      0.18      2149
         1.0       0.89      0.54      0.67     17116

    accuracy                           0.53     19265
   macro avg       0.50      0.50      0.43     19265
weighted avg       0.80      0.53      0.62     19265



In [42]:
from sklearn.metrics import f1_score
f1_score(y_test, clf.predict(X_test))

0.6741564889645925

In [43]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, clf.predict(X_test))

array([[ 973, 1176],
       [7815, 9301]])